In [24]:
import pandas as pd

# Biblioteca de Similaridade por cosseno https://pt.wikipedia.org/wiki/Similaridade_por_cosseno
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
QUANTIDADE_MINIMA_AVALIACOES_USUARIOS = 100
QUANTIDADE_MINIMA_AVALIACOES_LIVROS = 20

QUANTIDADE_RECOMENDACOES_COLABORATIVAS = 5


In [12]:
users_df = pd.read_parquet(r"backend\data\users.parquet")

In [13]:
paises = (
    users_df[["COUNTRY"]]
    .drop_duplicates()
    .reset_index()
    .to_parquet(r"backend\data\paises.parquet")
)

In [3]:
users_df = pd.read_parquet(r"data\users.parquet")
books_df = pd.read_parquet(r"data\books.parquet")
ratings_df = pd.read_parquet(r"data\ratings.parquet")

In [4]:
# Juntando a informação de avaliação com os detalhes do livro
avaliacoes_com_info_df = ratings_df.merge(books_df, on="ISBN")

In [7]:
considerar_usuarios_sr = (
    avaliacoes_com_info_df.groupby("USER_ID").count()["RATING"]
    > QUANTIDADE_MINIMA_AVALIACOES_USUARIOS
)
lista_usuarios_validos = considerar_usuarios_sr[considerar_usuarios_sr].index
usuarios_filtrados_df = avaliacoes_com_info_df[
    avaliacoes_com_info_df.USER_ID.isin(lista_usuarios_validos)
]
usuarios_filtrados_df.shape

(591896, 10)

In [8]:
considerar_livros_sr = (
    usuarios_filtrados_df.groupby("TITLE").count()["RATING"]
    > QUANTIDADE_MINIMA_AVALIACOES_LIVROS
)
lista_livros_validos = considerar_livros_sr[considerar_livros_sr].index
filtrados_df = usuarios_filtrados_df[
    usuarios_filtrados_df.TITLE.isin(lista_livros_validos)
]
filtrados_df.shape

(181098, 10)

In [14]:
# Criação de tabela pivotada com os livros como linhas, os usuários como colunas e as avaliações como valores
pivot_table_df = filtrados_df.pivot_table(
    index="TITLE", columns="USER_ID", values="RATING"
)
pivot_table_df.fillna(0, inplace=True)
pivot_table_df.to_parquet(r"data/pivot.parquet")

In [20]:
books_df = pd.read_parquet(r"data\books.parquet")

In [21]:
pivot_table_df = pd.read_parquet(r"data/pivot.parquet")

In [22]:
# Criação da matriz de similaridade por cosseno
similaridade_cosseno = cosine_similarity(pivot_table_df)
similaridade_cosseno.shape


(3810, 3810)

In [25]:
def livros_recomendados_similaridade(
    title: str, quant: int = QUANTIDADE_RECOMENDACOES_COLABORATIVAS
):
    books_df = pd.read_parquet(r"data\books.parquet")
    pivot_table_df = pd.read_parquet(r"data\pivot.parquet")
    similaridade_cosseno = cosine_similarity(pivot_table_df)

    try:
        if not title in pivot_table_df.index:
            return {
                "Success": False,
                "Message": f'O livro "{title}" não consta em nossa lista.',
            }

        index = np.where(pivot_table_df.index == title)[0][0]

        # Lista os items similares ao informado (começando do segundo índice para não pegar o próprio livro)
        similar_items = sorted(
            enumerate(similaridade_cosseno[index]), key=lambda x: x[1], reverse=True
        )[1 : quant + 1]

        data = []
        for i in similar_items:
            item = {}
            temp_df = books_df[books_df["TITLE"] == pivot_table_df.index[i[0]]]
            item["TITLE"] = list(temp_df.drop_duplicates("TITLE")["TITLE"].values)[0]
            item["AUTHOR"] = list(temp_df.drop_duplicates("TITLE")["AUTHOR"].values)[0]
            item["IMAGE_S"] = list(temp_df.drop_duplicates("TITLE")["IMAGE_S"].values)[
                0
            ]
            item["IMAGE_M"] = list(temp_df.drop_duplicates("TITLE")["IMAGE_M"].values)[
                0
            ]
            item["IMAGE_L"] = list(temp_df.drop_duplicates("TITLE")["IMAGE_L"].values)[
                0
            ]
            data.append(item)

        return {"success": True, "livros": data}
    except Exception as ex:
        return {"success": False, "Message": ex}

In [26]:
livros_recomendados_similaridade("10 Lb. Penalty")

{'Success': True,
 'livros': [{'TITLE': 'Danger',
   'AUTHOR': 'Dick Francis',
   'IMAGE_S': 'http://images.amazon.com/images/P/0449202631.01.THUMBZZZ.jpg',
   'IMAGE_M': 'http://images.amazon.com/images/P/0449202631.01.MZZZZZZZ.jpg',
   'IMAGE_L': 'http://images.amazon.com/images/P/0449202631.01.LZZZZZZZ.jpg'},
  {'TITLE': 'A Season in Purgatory',
   'AUTHOR': 'Dominick Dunne',
   'IMAGE_S': 'http://images.amazon.com/images/P/0345430557.01.THUMBZZZ.jpg',
   'IMAGE_M': 'http://images.amazon.com/images/P/0345430557.01.MZZZZZZZ.jpg',
   'IMAGE_L': 'http://images.amazon.com/images/P/0345430557.01.LZZZZZZZ.jpg'},
  {'TITLE': 'Die for Love',
   'AUTHOR': 'Elizabeth Peters',
   'IMAGE_S': 'http://images.amazon.com/images/P/0380731169.01.THUMBZZZ.jpg',
   'IMAGE_M': 'http://images.amazon.com/images/P/0380731169.01.MZZZZZZZ.jpg',
   'IMAGE_L': 'http://images.amazon.com/images/P/0380731169.01.LZZZZZZZ.jpg'},
  {'TITLE': 'Decider',
   'AUTHOR': 'Dick Francis',
   'IMAGE_S': 'http://images.amazon